In [1]:
from keras.preprocessing.text import Tokenizer
import numpy as np
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk import WordPunctTokenizer
import numpy as np 
import re
import pandas as pd
import fasttext
import fasttext.util
import random
from itertools import permutations,combinations
from numpy.linalg import norm


### fastText

In [2]:
ft = fasttext.load_model(r'cc.tr.300.bin')


In [3]:
df = pd.read_csv("words.txt")
liste=np.array(df).flatten()

In [4]:
def get_word_vectors(word_list, ft):
    """
    Get word vectors of a word list from a fasttext object
    """
    return np.array(list(map(ft.get_word_vector, word_list)))

In [5]:
vectors = get_word_vectors(liste, ft)
vectors

array([[-0.0167658 ,  0.06956569,  0.00177052, ..., -0.01685591,
        -0.00288318, -0.04962941],
       [-0.10739773,  0.05089148, -0.0300414 , ...,  0.12645064,
         0.07077996,  0.04066509],
       [-0.07227466,  0.01381767, -0.02900943, ...,  0.00052213,
         0.02655633,  0.01248306],
       ...,
       [-0.00151486, -0.00429392, -0.05861315, ..., -0.00267361,
         0.01220691, -0.00468195],
       [-0.02242724, -0.0397985 ,  0.03453791, ...,  0.03324145,
         0.08552809, -0.02271634],
       [-0.06946657, -0.01242691, -0.02154474, ..., -0.0037286 ,
         0.01136935,  0.01876289]], dtype=float32)

In [6]:
def cos_similarity_w_matrix(vector, matrix):
    """
    Cosine similarity of a vector to all rows of matrix
    """
    return np.dot(vector, matrix.T) / (np.linalg.norm(vector) * np.linalg.norm(matrix, axis=1))

In [7]:
def cos_sim_matrix(matrix):
    """
    Pairwise cosine similarity of all rows of matrix
    """
    cs = lambda inp : cos_similarity_w_matrix(inp, matrix)
    return np.array(list(map(cs, matrix)))

In [8]:
word_distances = cos_sim_matrix(vectors)
word_distances

array([[ 1.        ,  0.21145576,  0.04731809, ...,  0.00837442,
         0.23110276, -0.02808748],
       [ 0.21145576,  1.0000001 ,  0.03336043, ...,  0.06908277,
         0.20224693,  0.01267618],
       [ 0.04731809,  0.03336043,  1.        , ...,  0.26436856,
         0.22031526,  0.26049113],
       ...,
       [ 0.00837442,  0.06908277,  0.26436856, ...,  1.0000001 ,
         0.18734314,  0.1611891 ],
       [ 0.23110273,  0.20224692,  0.22031525, ...,  0.18734312,
         1.0000001 ,  0.03243906],
       [-0.02808748,  0.01267618,  0.26049113, ...,  0.1611891 ,
         0.03243906,  0.99999994]], dtype=float32)

In [39]:
word_distances[0].min()

-0.07205139

In [9]:
def find_furthest(k,word_distances,liste):
    indices = np.apply_along_axis(lambda x: np.argpartition(x, k)[:k], 1, word_distances)
    distant_words = np.apply_along_axis(lambda x: liste[x], 1, indices)
    #return distant_words
    dist_words_dict = dict(zip(liste, distant_words.tolist()))
    return distant_words.tolist()


In [10]:
#get nearest
def find_nearest(k,word_distances,liste):
    indices = np.apply_along_axis(lambda x: np.argsort(x, axis=0)[-k:], 1, word_distances)
    distant_words = np.apply_along_axis(lambda x:liste[x], 1, indices)
    dist_words_dict = dict(zip(liste, distant_words.tolist()))
    return distant_words.tolist()
    #return dist_words_dict


In [11]:
find_nearest(5,word_distances,liste)

{'abajur': ['mobilya', 'yastık', 'saksı', 'lamba', 'abajur'],
 'abaküs': ['kerpeten', 'çubuk', 'satranç', 'matematik', 'abaküs'],
 'abanma': ['ağırlık', 'tekme', 'üşenme', 'acıkma', 'abanma'],
 'abartı': ['espri', 'haksızlık', 'ayıp', 'yalan', 'abartı'],
 'acemi': ['cesur', 'nişancı', 'gönüllü', 'asker', 'acemi'],
 'acele': ['ihmal', 'çabuk', 'ısrar', 'telaş', 'acele'],
 'acente': ['otel', 'seyahat', 'bayi', 'turizm', 'acente'],
 'acıkma': ['uyku', 'sinirlilik', 'açlık', 'tokluk', 'acıkma'],
 'acıma': ['acılık', 'sevgi', 'vicdan', 'şefkat', 'acıma'],
 'acılık': ['hüzün', 'acıma', 'sıkıntı', 'lezzet', 'acılık'],
 'aciliyet': ['sorun', 'tuhaflık', 'tehlike', 'önem', 'aciliyet'],
 'açlık': ['sefalet', 'kıtlık', 'tokluk', 'yoksulluk', 'açlık'],
 'adalet': ['barış', 'ahlak', 'vicdan', 'hukuk', 'adalet'],
 'adak': ['dua', 'çömlek', 'mezar', 'ibadet', 'adak'],
 'adam': ['arkadaş', 'patron', 'kız', 'kadın', 'adam'],
 'adapte': ['çabuk', 'aşina ', 'alışkın', 'uyum', 'adapte'],
 'adayış': ['inan

In [12]:
def cosine_sim(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_1 = np.linalg.norm(vector1)
    norm_2 = np.linalg.norm(vector2)
    return dot_product / (norm_1 * norm_2)

In [13]:
p=find_furthest(5,word_distances,liste)


In [14]:
def furtherest_ones(index,liste,word_distances,list_length):
    temp=[]
    order=0

    for i in range(1000):
        if len(temp)==list_length:
            break
        index=np.argpartition(word_distances[index], order)[order] #returns num
        if not liste[index]  in temp:
            order +=1
            temp.append(liste[index]) #returns string              
                
    return temp
#Returns sequential the most distant ones

In [317]:
import math

In [318]:
print (math.comb(10, 2))


45


In [411]:
def random_dist_creator(word_distances,liste,list_length):
    temp9=[]

    temp9.append(liste[np.argpartition(word_distances[random.randint(0,980)],
                                                                random.randint(0,5))[random.randint(0,5)]])

    #if (0.15 > cosine_sim(ft[temp9[-1]],ft[liste[np.argpartition(word_distances[random.randint(0,980)],random.randint(0,10))[random.randint(0,10)]]])):
        
    #temp9.append(liste[np.argpartition(word_distances[random.randint(0,980)],random.randint(0,10))[random.randint(0,10)]])
                                    
        #if len(temp9)==2:
            
    while True:
        if results(temp9)==(math.comb(list_length, 2)):
            break
        a=random.randint(0,980)
        b=random.randint(0,5)
        index=np.argpartition(word_distances[a], b)[b]          
                #x=cosine_sim(ft[temp9[-1]],ft[temp9[-2]])
        y=cosine_sim(ft[temp9[-1]],ft[liste[index]])


                #if x>y and 0.16>y and 0.16>x and (not liste[index]  in temp9): 
        if 0.16>y and (not liste[index]  in temp9): 
            temp9.append(liste[index])
    return temp9


    
#Returns random sequential distant elements according to some restrictions


In [418]:
print(random_dist_creator(word_distances,liste,10))
print(random_dist_creator(word_distances,liste,10))
print(random_dist_creator(word_distances,liste,10))
print(random_dist_creator(word_distances,liste,10))
print(random_dist_creator(word_distances,liste,10))


['acıma', 'terlik', 'hayret', 'iğde', 'kuşku', 'güney', 'gün', 'hısım', 'grev', 'yüce']
['acente', 'ibrik', 'öncülük', 'soytarı', 'kucak', 'tiryaki', 'kuzgun', 'cılık', 'zakkum', 'ibre']
['acılık', 'ibre', 'aforoz', 'yorum', 'bakır', 'gezgin', 'çeyiz', 'oynaş', 'kepenk', 'uygunluk']
['tekel', 'alıntı', 'imza', 'ibrik', 'izlenim', 'ilave', 'gezegen', 'abaküs', 'örtü', 'gezgin']
['harita', 'nakavt', 'tabure', 'haksızlık', 'yağış', 'manken', 'ibraz', 'iğde', 'abaküs', 'dizgin']


In [416]:
u=random_dist_creator(word_distances,liste,10)
u

['avare',
 'ayıp',
 'röfle',
 'cılık',
 'muhbir',
 'payda',
 'gezgin',
 'aforoz',
 'uğultu',
 'acente']

In [417]:
cosine_sim(ft["röfle"],ft["uğultu"])

0.0088122655

In [410]:
def results(u):
    #temp1=[]
    res1=[]
    for a,b in combinations(u,2):
        if cosine_sim(ft[a],ft[b])<0.5:
            #temp1.append(cosine_sim(ft[a],ft[b]))#similarity remove_less_than(10,980)
            res1.append((a,b))
    return len(res1)
    #return dict(zip(res1,temp1))


## TEK KELİME İÇİN GEÇERLİ LİSTE

In [14]:
def find_furthest(k,word_distances,liste):
    indices = np.apply_along_axis(lambda x: np.argpartition(x, k)[:k], 1, word_distances)
    distant_words = np.apply_along_axis(lambda x:liste[x], 1, indices)
    dist_words_dict = dict(zip(liste, distant_words.tolist()))
    #return distant_words.tolist()
    return dist_words_dict

In [15]:
q=find_nearest(5,word_distances,liste)

In [263]:
def interitems(p,u,k): #interiteme condition koy ve replace edip tekrar çalıştır??
    temp=[]
    temp1=[]
    res1=[]
    for j in range(k):
        temp.append(ft.get_word_vector(p[u][j]))
    for a,b in combinations(temp,2):
        temp1.append(cosine_sim(a,b))#similarity remove_less_than(10,980)
    for x,y in combinations(p[u],2):
        res1.append((x,y))
    return(dict(zip(res1,temp1)))


In [264]:
final=[(liste[x],interitems(p,x,5)) for x in range(len(liste))]

In [261]:
final

[('abajur',
  {('küfür', 'sadakat'): 0.27324307,
   ('küfür', 'acılık'): 0.25565886,
   ('küfür', 'zayıflık'): 0.22034858,
   ('küfür', 'şiddetli'): 0.318791,
   ('sadakat', 'küfür'): 0.27324307,
   ('sadakat', 'acılık'): 0.2399291,
   ('sadakat', 'zayıflık'): 0.334805,
   ('sadakat', 'şiddetli'): 0.17417525,
   ('acılık', 'küfür'): 0.25565886,
   ('acılık', 'sadakat'): 0.2399291,
   ('acılık', 'zayıflık'): 0.39212072,
   ('acılık', 'şiddetli'): 0.2600376,
   ('zayıflık', 'küfür'): 0.22034858,
   ('zayıflık', 'sadakat'): 0.334805,
   ('zayıflık', 'acılık'): 0.39212072,
   ('zayıflık', 'şiddetli'): 0.34017813,
   ('şiddetli', 'küfür'): 0.318791,
   ('şiddetli', 'sadakat'): 0.17417525,
   ('şiddetli', 'acılık'): 0.2600376,
   ('şiddetli', 'zayıflık'): 0.34017813}),
 ('abaküs',
  {('koruma', 'iklim'): 0.33657178,
   ('koruma', 'temas'): 0.27469516,
   ('koruma', 'gizlilik'): 0.46701413,
   ('koruma', 'atmosfer'): 0.20966467,
   ('iklim', 'koruma'): 0.33657178,
   ('iklim', 'temas'): 0.234

In [818]:
# condition=less_than, higher_than
def condition_list(list_length,liste,condition,threshold,sub_group_length):
    temp=[]
    temp2=[]
    i=0
    while i<list_length:
        for key, value in liste[i][1].items(): # Again, iteritems() for memory-efficiency
            temp.append([key, value])
        i= i+1
    if(condition=="less_than"):
        for i in range(len(temp)):
            if temp[i][1]<threshold:
                temp2.append(temp[i])
            else:
                temp2.append("-1")
        temp2= [temp2[n:n+sub_group_length] for n in range(0, len(temp2), sub_group_length)]        
        return temp2  
    if(condition=="higher_than"):
        for i in range(len(temp)):
            if temp[i][1]>threshold:
                temp2.append(temp[i])
            else:
                temp2.append("-1")
                #en çok # else'e eklenecek
        temp2= [temp2[n:n+sub_group_length] for n in range(0, len(temp2), sub_group_length)]
        return temp2        

In [819]:
w=condition_list(981,final,"less_than",0.15,10)


In [811]:
def remove_less_than(sub_group_length,index):
 #liste uzunluğu
    indices = np.apply_along_axis(lambda x: np.argpartition(x, sub_group_length)[:sub_group_length], 1, word_distances)
    distant_words = np.apply_along_axis(lambda x:liste[x], 1, indices)
    print(distant_words[index])
    return(distant_words[index][-1]) 

In [812]:
remove_less_than(10,980)

['iğde' 'vitrin' 'manken' 'abajur' 'fındık' 'röfle' 'tütün' 'ebleh'
 'zakkum' 'abaküs']


'abaküs'

In [813]:
def remove_higher_than(sub_group_length,index):
 #liste uzunluğu
    indices = np.apply_along_axis(lambda x: np.argsort(x, axis=0)[-sub_group_length:], 1, word_distances)
    distant_words = np.apply_along_axis(lambda x:liste[x], 1, indices)
    print(distant_words[index])
    return(distant_words[index][0])


In [816]:
remove_higher_than(5,0)

['mobilya' 'yastık' 'saksı' 'lamba' 'abajur']


'mobilya'

In [ ]:
#define threshold condition which eliminate items that don't suit the condition and replace ones which suit the condition.
#Finally create lists with shuffled elements

## Semantic Tool Class (STEM-T=> Semantic Tool for Episodic Memory Task)

In [194]:
i=2#değişecek index
sub_group_length=5 #liste uzunluğu
index=0
remove_less_than(5,6)

'aciliyet'

Semantic item class will include turn word to vector (get_word_vectors) cos similiarities (cos_similarity_w_matrix, cos_sim_matrix,cos_sim_matrix, cosine_sim) and furthest elements (find_furthest) and nearest element (find_nearest)

In [ ]:
class semantic_tool(object):
    def __init__(self, word_list,ft, liste,matrix,k,x,y,p,u):
        self.age = age
        self.name = name
    def set_age(self, age):
        self.age = age
    def get_age(self):
        return self.age
    def age_diff(self, other):
        diff = self.age - other.age
        return abs(diff)
    
    def get_word_vectors(word_list, ft):
        """
        Get word vectors of a word list from a fasttext object
        """
        return np.array(list(map(ft.get_word_vector, word_list)))
    
    def cos_similarity_w_matrix(vector, matrix):
        """
        Cosine similarity of a vector to all rows of matrix
        """
        return np.dot(vector, matrix.T) / (np.linalg.norm(vector) * np.linalg.norm(matrix, axis=1))
    def cos_sim_matrix(matrix):
        """
        Pairwise cosine similarity of all rows of matrix
        """
        cs = lambda inp : cos_similarity_w_matrix(inp, matrix)
        return np.array(list(map(cs, matrix)))
    def find_furthest(k,word_distances,liste):
        indices = np.apply_along_axis(lambda x: np.argpartition(x, k)[:k], 1, word_distances)
        distant_words = np.apply_along_axis(lambda x:liste[x], 1, indices)
        dist_words_dict = dict(zip(liste, distant_words.tolist()))
        return distant_words.tolist()
        #return dist_words_dict
        
    def find_nearest(k,word_distances,liste):
        indices = np.apply_along_axis(lambda x: np.argsort(x, axis=0)[-k:], 1, word_distances)
        distant_words = np.apply_along_axis(lambda x:liste[x], 1, indices)
        dist_words_dict = dict(zip(liste, distant_words.tolist()))
        return distant_words.tolist()
        #return dist_words_dict

        
        #return dist_words_dict
    def cosine_sim(x, y):
        return sum(x * y) / (sum(x**2) * sum(y**2)) ** 0.5
    def interitems(p,u,k):
        temp=[]
        temp1=[]
        res1=[]
        for j in range(k):
            temp.append(ft.get_word_vector(p[u][j]))
        for a,b in combinations(temp,2):
            temp1.append(cosine_sim(a,b))
        for a,b in combinations(p[u],2):
            res1.append((a,b))
        return(dict(zip(res1,temp1)))
    def crate_lists():
        return("-1")

In [ ]:
p2 = Person(26, "Emily")
print(p2.get_age())
p2 = Person(26, "Emily")
print(Person.get_age(p2))